In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -qq langdetect
from langdetect import detect
import string
!pip install -qq contractions
!pip install -qq demoji
import demoji
demoji.download_codes()
!pip install -qq datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.6 MB/s eta 0:00:00


<ipython-input-3-28a5514cb77a>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00


In [2]:
!pip install -qq inltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.1/354.1 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
header_names = ['label', 'text']
df_train = pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/tamil_offensive_full.csv", sep='\t',names=header_names)
df_train = df_train.dropna(how='all')
df_train = pd.DataFrame(df_train)
df_train

,label,text
0,Offensive_Untargetede,Lighta adanga maaru bgm maathiri theriyuthu
1,Offensive_Untargetede,Sema...last dialogue awesome....sanda pottu ed...
2,Offensive_Targeted_Insult_Other,Nice But pesura dialogue onnum puriyala romba...
3,Not_offensive,BGM asusual therikka vitaru Sam cs
4,Not_offensive,Entha Diwali Kaithi Diwali
...,...,...
43914,Not_offensive,கவுண்டர் சமுதாய மக்கள் சார்பாக வெற்றி பெற வாழ்...
43915,Offensive_Targeted_Insult_Group,Comment la periya pudungi madiri jaadi name ah...
43916,Not_offensive,திரைப்படம் பொங்கலுக்கு ரிலீசானால் நன்றாக இருக்...
43917,Not_offensive,Sathi verundri valara itha Padam oru eduthukattu


In [ ]:
label_mapping = {
    'Not_offensive': 0,
    'Offensive_Untargetede': 1,
    'Offensive_Targeted_Insult_Individual': 1,
    'Offensive_Targeted_Insult_Group': 1,
    'Offensive_Targeted_Insult_Other': 1,
    'not-Tamil': 2,
}
df_train['label'].replace(label_mapping, inplace=True)
print(df_train.info())
label_counts = df_train['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43919 entries, 0 to 43918
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   43919 non-null  int64 
 1   text    43919 non-null  object
dtypes: int64(1), object(1)
memory usage: 686.4+ KB
None
0    31808
1    10325
2     1786
Name: label, dtype: int64


In [ ]:
df_train.head(20)

,label,text
0,1,Lighta adanga maaru bgm maathiri theriyuthu
1,1,Sema...last dialogue awesome....sanda pottu ed...
2,1,Nice But pesura dialogue onnum puriyala romba...
3,0,BGM asusual therikka vitaru Sam cs
4,0,Entha Diwali Kaithi Diwali
5,1,"Trailer nalla dhan iruku, theva illama en #BIG..."
6,1,Bigil kuda realsee panatha da unga padam odathu
7,0,Waiting for lokesh kanakaraj Thalapathy 64 Th...
8,1,Indha padathoda trailer eh indha level la iruk...
9,0,Screenplay vera level laaa iruku.... Waiting t...
